# This notebook is for the data collection Date - 9/04/20
--------

The below code cell is for the imports we make two imports only

1. praw - for scrapping reddit data
2. pandas - for storing data as csv

In [2]:
import praw
import pandas as pd
import string
import datetime as dt
from praw.models import MoreComments
import re 

This cell makes an reddit object to connect to the reddit api and this will be used to search and find the subreddits of our choice. The config dictionary contains the parameters needed to connect to the reddit api

1. client_id
2. client_secret
3. user_agent
4. username
5. password

In [3]:
config = {
    'client_id':'paPOXE0wodYqVg',
    'client_secret':'e-ciuNWTuEHcO4GY-ZWlovGahT0',
    'user_agent':'data_collection',
    'user_name':'anuj_2110',
    'password':'21101998@anj'}

reddit  = praw.Reddit(client_id = config['client_id'],
                      client_secret = config['client_secret'],
                      user_agent = config['user_agent'],
                      user_name = config['user_name'],
                      password = config['password'])

Defining variables and functions for data collection purposes

In [7]:
def clean_text(text): 
    '''
    function for cleaning the text
    '''
    text = re.sub(r"http\S+", " ", text)
    result = re.sub('[^A-Za-z0-9]+', ' ', text)
    return result.lower()
def time_from_timestamp(timestamp):
    '''
    function for converting the timestamp to time
    '''
    return dt.datetime.fromtimestamp(timestamp)

def retrieve_data(reddit):
    '''
    This function is for retrieving data from the reddit api,cleaning and storing it
    '''
    flairs = ['AskIndia','Non-Political','Scheduled','Photography','Coronavirus',
          'Science/Technology','Politics','Business/Finance','Policy/Economy','Sports','Food'] # a list for flairs which will be used for searching by flairs
    data = {
             'id': [],
             'flair' : [],
             'title' : [],
             'body'  : [],
             'num_cmnts' : [],
             'created': [],
             'score': [],
             'comments': []} #we will store the data in this dictionary and afterwards use it to store it as pandas dataframe

    df_file = "../reddit_flair_data.csv" #file for saving data

    subred = reddit.subreddit('india') # making the subreddit object for india
    
    for flair in flairs:
        '''
        here we parse through each flair and we try to search for subreddits for one flair at a time and store them
        '''
        subreds = subred.search(flair, limit=500)
        for submission in subreds:
            data["flair"].append(flair)
            data["title"].append(submission.title)
            data["score"].append(submission.score)
            data["id"].append(submission.id)
            data["num_cmnts"].append(submission.num_comments)
            data["created"].append(submission.created)
            data["body"].append(submission.selftext)
            comment = ''
            for top_level_comment in submission.comments:
                if isinstance(top_level_comment, MoreComments):
                    continue
                comment = comment + ' ' + top_level_comment.body
            data["comments"].append(comment)
        print(flair+" done")
        
    df = pd.DataFrame(data)
    
    df['created'] = df['created'].apply(time_from_timestamp)
    df['title'] = df['title'].apply(clean_text)
    df['body'] =  df['body'].apply(clean_text)
    df['comments'] = df['comments'].apply(clean_text)
    
    df.to_csv(df_file,index=False)
    return df

In [9]:
df = retrieve_data(reddit)

AskIndia done
Non-Political done
Scheduled done
Photography done
Coronavirus done
Science/Technology done
Politics done
Business/Finance done
Policy/Economy done
Sports done
Food done
